In [1]:
from phi3v import FinetunePhi3V

# Sample data
data = [
    {
        "image": "image.jpg",
        "input": "Describe the image.",
        "output": "This is a picture of two bags."
    },
    {
        "image": "image.jpg",
        "input": "Describe the image.",
        "output": "This is a picture of two bags."
    },
        {
        "image": "image.jpg",
        "input": "Describe the image.",
        "output": "This is a picture of two bags."
    },
            {
        "image": "image.jpg",
        "input": "Describe the image.",
        "output": "This is a picture of two bags."
    },
    # Add more data points as needed
]

# Initialize the finetune class with the data
finetuner = FinetunePhi3V(
    data=data,
    epochs=3,
    learning_rate=5e-5,
    warmup_ratio=0.1,
    gradient_accumulation_steps=32,
    optim="adamw_hf",
    model_id="microsoft/Phi-3-vision-128k-instruct",
    peft_r=16,
    peft_alpha=32,
    peft_dropout=0.1,
    formatter="<|user|>\n<|image_1|>{prompt}<|end|><|assistant|>{answer}<|end|>"
)

# Run the finetuning process
finetuner.run()


2025-01-16 13:29:41.004041: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-16 13:29:41.035713: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-16 13:29:41.035742: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-16 13:29:41.036650: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-16 13:29:41.042258: I tensorflow/core/platform/cpu_feature_guar

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[2025-01-16 13:30:23,480] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/cqilab/anaconda3/envs/sgrs/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/cqilab/anaconda3/envs/sgrs/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status
/home/cqilab/anaconda3/envs/sgrs/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You are not running the flash-attention implementation, expect numerical differences.
/home/cqilab/anaconda3/envs/sgrs/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization

Step,Training Loss
1,1.511300
2,1.511300
3,1.492300


[1/3, Epoch 1.0] Step	Training Loss: 1.5113
[2/3, Epoch 2.0] Step	Training Loss: 1.5113
[3/3, Epoch 3.0] Step	Training Loss: 1.4923
[3/3, Epoch 3.0] Step	Training Loss: N/A
